In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import import_ipynb
from rotation import rotation_matrix, unit_vector, angle_between, x_rotation, y_rotation, z_rotation

# for machine learning
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import TomekLinks, RandomUnderSampler, EditedNearestNeighbours, AllKNN

import csv
from csv import reader
import pickle
from collections import Counter
from matplotlib import pyplot

importing Jupyter notebook from rotation.ipynb


In [2]:
NUM_FEATURES = 3
NUM_JOINTS = 20
NUM_FRAMES = 16
FILE_NAME = '../train.csv'
test_FILE_NAME = '../test.csv'

In [3]:
dtf = pd.read_csv(FILE_NAME, header = None)

X = dtf.iloc[:,:-1]
y = dtf.iloc[:,-1]

dtf_test = pd.read_csv(test_FILE_NAME, header = None)
#dtf_test = dtf_test.set_index(dtf_test.columns[0])

X_test = dtf_test.iloc[:,:]
X_test_index = dtf_test.iloc[:,0]

In [10]:
def read_xyz(row):
    skeleton_data, label = read_skeleton(row)
    
    data = np.zeros((NUM_FRAMES, NUM_JOINTS, NUM_FEATURES))
    for m, i in enumerate(skeleton_data['frame_info']):
        for n, j in enumerate(i['joint_info']):
            feature_info = j['feature_info']
            data[m, n, :] = [feature_info['x'], feature_info['y'], feature_info['z']]

    # data = data.transpose(2, 0, 1)
    return data, label

In [11]:
def read_skeleton(row):
    label = row[-1]
    #label = -1
    skeleton_data = {}
    skeleton_data['index'] = row[0]
    skeleton_data['num_frame'] = NUM_FRAMES
    skeleton_data['frame_info'] = []
    
    for frame in range(NUM_FRAMES):
        offset = NUM_JOINTS * NUM_FEATURES
        data_in_frame = row[frame*offset:(frame+1)*offset]
        frame_info = {}
        frame_info['num_joints'] = NUM_JOINTS
        frame_info['joint_info'] = []
        
        for feature in range(NUM_JOINTS):
            offset = NUM_FEATURES
            data_in_joint = data_in_frame[feature*offset:(feature+1)*offset]
            joint_info = {}
            joint_info['num_features'] = NUM_FEATURES
            joint_info['feature_info'] = {
                k: float(v)
                for k, v in zip(['x', 'y', 'z'], data_in_joint)
            }
            frame_info['joint_info'].append(joint_info)
                    
        skeleton_data['frame_info'].append(frame_info)
    return skeleton_data, label

In [16]:
# create train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    stratify=np.array(y), 
                                                    random_state=42)

# create train and validation datasets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.15, 
                                                  stratify=np.array(y_train), 
                                                  random_state=42)


print('Initial Dataset Size:', X.shape)
print('Initial Train and Test Datasets Size:', X_train.shape, X_test.shape)
print('Train and Validation Datasets Size:', X_train.shape, X_val.shape)
print('Train, Test and Validation Datasets Size:', X_train.shape, X_test.shape, X_val.shape)

Initial Dataset Size: (9388, 961)
Initial Train and Test Datasets Size: (5585, 961) (2817, 961)
Train and Validation Datasets Size: (5585, 961) (986, 961)
Train, Test and Validation Datasets Size: (5585, 961) (2817, 961) (986, 961)


In [14]:
fp = np.zeros((len(X_train), NUM_FRAMES, NUM_JOINTS, NUM_FEATURES), dtype=np.float32) #construct a matrix, with num of data, num of features for each joint, num of frames, num of joints, num of people(always 1 in our case)
print(len(X_train))
for i, row in enumerate(X_train.to_numpy()):
    data, label = read_xyz(row)
    fp[i, :, :, :] = data
#fp = normalisation(fp)
# np.save('train_smote_custom.npy', fp)

7510


In [15]:
fp.shape

(7510, 16, 20, 3)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 32

# Create train generator.
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=30, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')
train_generator = train_datagen.flow(x_train, y_train_ohe, shuffle=False, 
                                     batch_size=BATCH_SIZE, seed=1)
                                     
# Create validation generator
val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = train_datagen.flow(x_val, y_val_ohe, shuffle=False, 
                                   batch_size=BATCH_SIZE, seed=1)       

In [ ]:
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the InceptionV3 model so we can do transfer learning
base_inception = InceptionV3(weights='imagenet', include_top=False, 
                             input_shape=(299, 299, 3))
                             
# Add a global spatial average pooling layer
out = base_inception.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
total_classes = y_train_ohe.shape[1]
predictions = Dense(total_classes, activation='softmax')(out)

model = Model(inputs=base_inception.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy']) 
model.summary()

In [ ]:
# labels = []
# for i, row in enumerate(y_train.values):
#     labels.append(row)
# with open('label_cnn.pkl', 'wb') as f:
#     pickle.dump(labels, f)